<a href="https://colab.research.google.com/github/M-110/testing-with-pytest-2/blob/main/02_Writing_Test_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!pip install pytest --upgrade -q

     |████████████████████████████████| 297 kB 5.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!wget https://media.pragprog.com/titles/bopytest2/code/bopytest2-code.zip -q
!unzip -q bopytest2-code.zip

In [4]:
%cd code/cards_proj

/content/code/cards_proj


In [8]:
!pip install tinydb -q
!pip install typer -q
!pip install rich -q

     |████████████████████████████████| 232 kB 5.1 MB/s 
     |████████████████████████████████| 51 kB 6.0 MB/s 


## Specify pythonpath from pytest.ini

In [9]:
%%writefile pytest.ini
[pytest]
pythonpath = src

Writing pytest.ini


In [5]:
!mkdir tests

# Basic tests

In [6]:
%%writefile tests/test_card.py
"""Test card data class"""
from cards import Card


def test_field_access():
    card = Card("something", "brian", "todo", 123)
    assert card.summary == "something"
    assert card.owner == "brian"
    assert card.state == "todo"
    assert card.id == 123


def test_defaults():
    card = Card()
    assert card.summary is None
    assert card.owner is None
    assert card.state == "todo"
    assert card.id is None


def test_equality():
    card1 = Card("something", "brian", "todo", 123)
    card2 = Card("something", "brian", "todo", 123)
    assert card1 == card2


def test_equality_with_different_ids():
    card1 = Card("something", "brian", "todo", 123)
    card2 = Card("something", "brian", "todo", 1234)
    assert card1 == card2


def test_from_dict():
    card_dict = dict(summary="something",
                     owner="brian",
                     state="todo",
                     id=123)
    card1 = Card("something", "brian", "todo", 123)
    card2 = Card.from_dict(card_dict)
    assert card1 == card2


def test_to_dict():
    card = Card("something", "brian", "todo", 123)
    expected_card_dict = dict(summary="something",
                              owner="brian",
                              state="todo",
                              id=123)
    actual_card_dict = card.to_dict()
    assert expected_card_dict == actual_card_dict


Writing tests/test_card.py


In [11]:
!python -m pytest -v

============================= test session starts ==============================
platform linux -- Python 3.7.13, pytest-7.1.2, pluggy-1.0.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/code/cards_proj, configfile: pytest.ini
plugins: typeguard-2.7.1
collected 6 items                                                              

tests/test_card.py::test_field_access PASSED                             [ 16%]
tests/test_card.py::test_defaults PASSED                                 [ 33%]
tests/test_card.py::test_equality PASSED                                 [ 50%]
tests/test_card.py::test_equality_with_different_ids PASSED              [ 66%]
tests/test_card.py::test_from_dict PASSED                                [ 83%]
tests/test_card.py::test_to_dict PASSED                                  [100%]

============================== 6 passed in 0.08s ===============================


# pytest.fail()

used as an assertion helper

In [12]:
%%writefile tests/test_fail.py
import pytest


def test_pytest_fail():
  pytest.fail("Oh no!")

Writing tests/test_fail.py


In [13]:
!pytest 

============================= test session starts ==============================
platform linux -- Python 3.7.13, pytest-7.1.2, pluggy-1.0.0
rootdir: /content/code/cards_proj, configfile: pytest.ini
plugins: typeguard-2.7.1
collected 7 items                                                              

tests/test_card.py ......                                                [ 85%]
tests/test_fail.py F                                                     [100%]

=================================== FAILURES ===================================
_______________________________ test_pytest_fail _______________________________

    def test_pytest_fail():
>     pytest.fail("Oh no!")
E     Failed: Oh no!

tests/test_fail.py:5: Failed
=========================== short test summary info ============================
FAILED tests/test_fail.py::test_pytest_fail - Failed: Oh no!
========================= 1 failed, 6 passed in 0.16s ==========================


In [46]:
%%writefile tests/test_fail_2.py
import pytest
from cards import Card


def assert_identical(c1, c2):
  __tracebackhide__ = True
  assert c1 == c2
  if c1.id != c2.id:
    pytest.fail(f"id's don't match. {c1.id} != {c2.id}")


def test_identical():
  c1 = Card("foo", id=123)
  c2 = Card("foo", id=124)
  assert_identical(c1, c2)

def test_identical_2():
  c1 = Card("foo", id=123)
  c2 = Card("foo", id=124)
  assert c1.id == c2.id

Writing tests/test_fail_2.py


In [41]:
!pytest -v

============================= test session starts ==============================
platform linux -- Python 3.7.13, pytest-7.1.2, pluggy-1.0.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/code/cards_proj, configfile: pytest.ini
plugins: typeguard-2.7.1
collected 9 items                                                              

test_fail_2.py::test_identical FAILED                                    [ 11%]
test_fail_2.py::test_identical_2 FAILED                                  [ 22%]
tests/test_card.py::test_field_access PASSED                             [ 33%]
tests/test_card.py::test_defaults PASSED                                 [ 44%]
tests/test_card.py::test_equality PASSED                                 [ 55%]
tests/test_card.py::test_equality_with_different_ids PASSED              [ 66%]
tests/test_card.py::test_from_dict PASSED                                [ 77%]
tests/test_card.py::test_to_dict PASSED                                  [ 88%]
tests/test_

# Expected exceptions

In [47]:
%%writefile tests/test_exceptions.py
import pytest

import cards

import pytest
import cards


def test_no_path_raises():
    with pytest.raises(TypeError):
        cards.CardsDB()


def test_raises_with_info():
    match_re = r"missing 1 .* positional argument"
    with pytest.raises(TypeError, match=match_re):
        cards.CardsDB()


def test_raises_with_info_alt():
    with pytest.raises(TypeError) as info:
        cards.CardsDB()
    expected = "missing 1 required positional argument"
    assert expected in str(info.value)


Writing tests/test_exceptions.py


In [48]:
!pytest tests/test_exceptions.py -v

============================= test session starts ==============================
platform linux -- Python 3.7.13, pytest-7.1.2, pluggy-1.0.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/code/cards_proj, configfile: pytest.ini
plugins: typeguard-2.7.1
collected 3 items                                                              

tests/test_exceptions.py::test_no_path_raises PASSED                     [ 33%]
tests/test_exceptions.py::test_raises_with_info PASSED                   [ 66%]
tests/test_exceptions.py::test_raises_with_info_alt PASSED               [100%]

============================== 3 passed in 0.06s ===============================


# Structuring Test Functions

**Arrange-Act-Assert**

or

**Given-When-Then**

1. Get ready to do something
2. Do something
3. Check that it worked

**Do NOT** interleave multiple assert patterns in a test function


In [49]:
%%writefile tests/test_gwt.py
from cards import Card


def test_to_dict():
  # Given a card with known contents and a dictionar with matching contents
  card = Card("something", "brian", "todo", 123)
  expected_card_dict = dict(summary="something",
                            owner="brian",
                            state="todo",
                            id=123)
  
  # When to_dict is called on the card
  actual_card_dict = card.to_dict()

  # Then the resulting dictionary will match the original contents
  assert expected_card_dict == actual_card_dict

Writing tests/test_gwt.py


# Grouping Tests with classes

In [52]:
%%writefile tests/test_classes.py
from cards import Card


class TestEquality:
  def test_equality(self):
    c1 = Card("a", "b", "c", 123)
    c2 = Card("a", "b", "c", 123)
    assert c1 == c2

  def test_equality_with_different_ids(self):
    c1 = Card("a", "b", "c", 123)
    c2 = Card("a", "b", "c", 124)
    assert c1 == c2

  def test_inequality(self):
    c1 = Card("a", "b", "c", 123)
    c2 = Card("a", "b", "d", 123)
    assert c1 != c2

Overwriting tests/test_classes.py


In [54]:
!pytest tests/test_classes.py -v

============================= test session starts ==============================
platform linux -- Python 3.7.13, pytest-7.1.2, pluggy-1.0.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/code/cards_proj, configfile: pytest.ini
plugins: typeguard-2.7.1
collected 3 items                                                              

tests/test_classes.py::TestEquality::test_equality PASSED                [ 33%]
tests/test_classes.py::TestEquality::test_equality_with_different_ids PASSED [ 66%]
tests/test_classes.py::TestEquality::test_inequality PASSED              [100%]

============================== 3 passed in 0.07s ===============================


# Running subset of tests

Subset | Syntax
--- | ---
Single test method | pytest path/test_module.py::TestClass::test_method
All tests in a class | pytest path/test_module.py::TestClass
Single test function | pytest path/test_module.py::test_function
All tests in a module | pytest path/test_module.py
All tests in a directory | pytest path
Tests matching a name pattern | pytest -k pattern
Tests by marker | Covered in Chapter 6, ​Markers​.